<a href="https://colab.research.google.com/github/swissecon-cryptecon/public_blockchains/blob/main/Public_Blockchains.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from messari.messari import Messari
import pandas as pd

In [ ]:
messari = Messari('OwwZ+mQ92hTCnYuzB-Gx9+927B2h-PNBIiyihjsVMmnF3to4')

In [ ]:
#list of variables, see also documentation: https://messari.io/api/docs#tag/News/operation/Get%20News%20for%20Asset
print(dir(messari)) 
list_messari_metrics = messari.list_asset_timeseries_metric_ids()
data = list_messari_metrics["data"]["metrics"]  
df = pd.DataFrame(data)

In [ ]:
assets = 'bitcoin'
metric = 'price'
start = '2020-06-01'
end = '2021-01-01'
timeseries_df = messari.get_asset_timeseries(asset_slugs=assets, asset_metric=metric, start=start, end=end)
timeseries_df

In [ ]:
print(dir(messari)) ### see documentation: https://messari.io/api/docs#tag/News/operation/Get%20News%20for%20Asset
list_messari_metrics = messari.list_asset_timeseries_metric_ids()
data = list_messari_metrics["data"]["metrics"]
df = pd.DataFrame(data)

In [ ]:
sply.circ